In [1]:
import os
import json
from math import ceil

from shutil import copyfile
from pathlib import Path
from scipy.io.wavfile import read as wavread

In [2]:
minutes = 2.5
new_folder = f"data/esd_vctk_{int(minutes)}"

In [3]:
!mkdir $new_folder

In [4]:
!cp -r data/esd_vctk_1/* $new_folder

In [5]:
with open("emo_speaker_file.json", "r") as f:
    emo_filelist = json.load(f)

In [6]:
time_to_add = (minutes - 1) * 60

In [7]:
speakers_to_ignore = ["0011", "0012", "0013", "0014", "0020"]

In [8]:
one_min_wavs = set()
one_min_wavs_path = Path("data/esd_vctk_1/resampled")
for speaker in speakers_to_ignore:
    one_min_wavs |= {file.stem for file in (one_min_wavs_path / speaker).rglob("*.wav")}

In [9]:
orig_wav_path = Path(r"data/esd_vctk/resampled")
speaker_added_minutes = {s: {emo: 0 for emo in emo_filelist.keys()} for s in speakers_to_ignore}
speaker_added_files = {s: [] for s in speakers_to_ignore}
c = 0
for speaker in speakers_to_ignore:
    path_to_speaker_wavs = orig_wav_path / speaker
    speaker_orig_wavs = {file.stem for file in path_to_speaker_wavs.rglob("*.wav")}
    for emo in emo_filelist.keys():
        files_to_check = [file for file in emo_filelist[emo][speaker] if file in speaker_orig_wavs]
        for file in files_to_check:
            if file not in one_min_wavs and speaker_added_minutes[speaker][emo] < time_to_add:
                sr, wav = wavread(path_to_speaker_wavs / f"{file}.wav")
                wav_time = wav.shape[0] / sr
                speaker_added_minutes[speaker][emo] += wav_time
                speaker_added_files[speaker].append(file)
            else:
                c += 1

In [10]:
orig_path = Path(r"data/esd_vctk")
copy_path = Path(new_folder)
for folder in os.listdir(orig_path):
    files_folder = os.listdir(orig_path / folder / "0017")
    ext = files_folder[0].split(".")[-1]
    for speaker, filelist in speaker_added_files.items():
        for file in filelist:
            copyfile(orig_path / folder / speaker / f"{file}.{ext}", copy_path / folder / speaker / f"{file}.{ext}")


In [11]:
speaker_time = 0
for speaker in speaker_added_files.keys():
    speaker_time = 0
    for file in (Path(new_folder) / "resampled" / speaker).rglob("*.wav"):
        sr, wav = wavread(file)
        wav_time = wav.shape[0] / sr
        speaker_time += wav_time
    print(speaker, round(speaker_time / 5 / 60, 1))

0011 2.7
0012 2.7
0013 2.7
0014 2.8
0020 2.8
